In [25]:
# 0801.py
import cv2
import numpy as np
#1
def findLocalMaxima(src):
    kernel= cv2.getStructuringElement(shape=cv2.MORPH_RECT, ksize=(11,11))
    dilate = cv2.dilate(src,kernel)# local max if kernel=None, 3x3 
    localMax = (src == dilate)
    
    erode = cv2.erode(src,kernel) #  local min if kernel=None, 3x3 
    localMax2 = src > erode      
    localMax &= localMax2
    points = np.argwhere(localMax == True)
    points[:,[0, 1]] = points[:,[1, 0]] # switch x, y 
    return points

#2
src = cv2.imread('./data/CornerTest.jpg')
gray = cv2.cvtColor(src, cv2.COLOR_BGR2GRAY)
res = cv2.preCornerDetect(gray, ksize=3)
ret, res2 = cv2.threshold(np.abs(res), 0.1, 0, cv2.THRESH_TOZERO)
corners = findLocalMaxima(res2)
print('corners.shape=', corners.shape)

#3
dst = src.copy()  
for x, y in corners:    
    cv2.circle(dst, (x, y), 5, (0,0,255), 2)
    
cv2.imshow('dst',  dst) 
cv2.waitKey()
cv2.destroyAllWindows()


corners.shape= (8, 2)


In [31]:
# 0802.py
import cv2
import numpy as np
#1
src = cv2.imread('./data/CornerTest.jpg')
gray = cv2.cvtColor(src, cv2.COLOR_BGR2GRAY)
res = cv2.cornerEigenValsAndVecs(gray, blockSize=5, ksize=3)
print('res.shape=', res.shape)
eigen = cv2.split(res)

#2
T = 0.2
ret, edge = cv2.threshold(eigen[0], T, 255, cv2.THRESH_BINARY)
edge = edge.astype(np.uint8)

#3
corners = np.argwhere(eigen[1]>T)
corners[:,[0, 1]] = corners[:,[1, 0]] # switch x, y
print('len(corners) =', len(corners))

dst = src.copy()
for x, y in corners:  
    cv2.circle(dst, (x, y), 5, (0,0,255), 2)
    
cv2.imshow('edge',  edge) 
cv2.imshow('dst',  dst)
cv2.waitKey()
cv2.destroyAllWindows()


res.shape= (512, 512, 6)
len(corners) = 8


In [29]:
cv2.imshow('src',src)
cv2.imshow('dst',dst)
cv2.waitKey()
cv2.destroyAllWindows()

In [33]:
# 0803.py
import cv2
import numpy as np

#1
src = cv2.imread('./data/CornerTest.jpg')
gray = cv2.cvtColor(src, cv2.COLOR_BGR2GRAY)
eigen = cv2.cornerMinEigenVal(gray, blockSize=5)
print('eigen.shape=', eigen.shape)

#2
T = 0.2
corners  = np.argwhere(eigen> T)
corners[:,[0, 1]] = corners[:,[1, 0]] # switch x, y
print('len(corners ) =', len(corners ))
dst = src.copy()
for x, y in corners :    
    cv2.circle(dst, (x, y), 3, (0,0,255), 2)
    
cv2.imshow('dst',  dst)
cv2.waitKey()
cv2.destroyAllWindows()


eigen.shape= (512, 512)
len(corners ) = 8


In [58]:
# 0804.py
import cv2
import numpy as np

#1
def findLocalMaxima(src):
    kernel= cv2.getStructuringElement(shape=cv2.MORPH_RECT, ksize=(11,11))
    dilate = cv2.dilate(src,kernel)# local max if kernel=None, 3x3 
    localMax = (src == dilate)
    
    erode = cv2.erode(src,kernel) #  local min if kernel=None, 3x3 
    localMax2 = src > erode      
    localMax &= localMax2
    points = np.argwhere(localMax == True)
    points[:,[0, 1]] = points[:,[1, 0]] # switch x, y
    return points
#2
src = cv2.imread('./data/CornerTest.jpg')
gray = cv2.cvtColor(src, cv2.COLOR_BGR2GRAY)
res = cv2.cornerHarris(gray, blockSize=5, ksize=3, k=0.01)
ret, res = cv2.threshold(np.abs(res),0.02, 0, cv2.THRESH_TOZERO)
res8 = cv2.normalize(res, None, 0, 255, cv2.NORM_MINMAX, dtype=cv2.CV_8U)
cv2.imshow('res8',  res8)

corners = findLocalMaxima(res)
print('corners=', corners)

#3
#corners = np.float32(corners).copy()
corners = corners.astype(np.float32, order='C')
term_crit = (cv2.TERM_CRITERIA_MAX_ITER+cv2.TERM_CRITERIA_EPS, 10, 0.01)
corners2 = cv2.cornerSubPix(gray, corners,(5,5),(-1,-1), term_crit)
print('corners2=', corners2)

dst = src.copy()
for x, y in corners2:    
    cv2.circle(dst, (x, y), 3, (0,0,255), 2)
cv2.imshow('dst',  dst)
cv2.waitKey()
cv2.destroyAllWindows()


corners= [[109 127]
 [264 127]
 [267 167]
 [386 170]
 [109 268]
 [167 271]
 [170 374]
 [386 374]]
corners2= [[107.559364 125.559456]
 [265.44077  125.559235]
 [265.55923  168.44174 ]
 [387.4408   168.55888 ]
 [107.558365 269.44095 ]
 [168.4415   269.55927 ]
 [168.55922  375.4408  ]
 [387.4408   375.4408  ]]


In [62]:
# 0805.py
import cv2
import numpy as np

#1
src = cv2.imread('./data/CornerTest.jpg')
gray = cv2.cvtColor(src, cv2.COLOR_BGR2GRAY)
res = cv2.cornerHarris(gray, blockSize=5, ksize=3, k=0.01)

#2
res = cv2.dilate(res, None) # 3x3 rect kernel
ret, res = cv2.threshold(res, 0.01*res.max(),255,cv2.THRESH_BINARY)
res8 = np.uint8(res)
cv2.imshow('res8',  res8)

#3
ret, labels, stats, centroids = cv2.connectedComponentsWithStats(res8)
print('centroids.shape=', centroids.shape)
print('centroids=',centroids)
centroids = np.float32(centroids)

#4
term_crit=(cv2.TERM_CRITERIA_MAX_ITER+cv2.TERM_CRITERIA_EPS,10, 0.001)
corners = cv2.cornerSubPix(gray, centroids, (5,5), (-1,-1), term_crit)
print('corners=',corners)

#5 
dst = src.copy()
for x, y in corners[1:]:    
    cv2.circle(dst, (x, y), 5, (0,0,255), 2)

cv2.imshow('dst',  dst) 
cv2.waitKey()
cv2.destroyAllWindows()


centroids.shape= (9, 2)
centroids= [[255.53481922 255.53107522]
 [108.         126.        ]
 [265.         126.        ]
 [266.         168.        ]
 [387.         169.        ]
 [108.         269.        ]
 [168.         270.        ]
 [169.         375.        ]
 [387.         375.        ]]
corners= [[255.53482  255.53108 ]
 [107.55841  125.55865 ]
 [265.44165  125.55835 ]
 [265.55832  168.44258 ]
 [387.44168  168.55795 ]
 [107.557495 269.44186 ]
 [168.4424   269.55838 ]
 [168.55832  375.44168 ]
 [387.44168  375.44168 ]]


In [64]:
# 0806.py
import cv2
import numpy as np

#1
src = cv2.imread('./data/CornerTest.jpg')
gray = cv2.cvtColor(src, cv2.COLOR_BGR2GRAY)

K = 5 
##K = 10
corners = cv2.goodFeaturesToTrack(gray, maxCorners=K,
              qualityLevel=0.05, minDistance=10)
print('corners.shape=',corners.shape)
print('corners=',corners)

#2
corners2 = cv2.goodFeaturesToTrack(gray, maxCorners=K,
               qualityLevel=0.05, minDistance=10,
               useHarrisDetector=True, k=0.04)
print('corners2.shape=',corners2.shape)
print('corners2=',corners2)

#3
dst = src.copy()
corners = corners.reshape(-1, 2)
for x, y in corners:    
    cv2.circle(dst, (x, y), 5, (0,0,255), -1)

corners2 = corners2.reshape(-1, 2)
for x, y in corners2:    
    cv2.circle(dst, (x, y), 5, (255,0,0), 2)
    
cv2.imshow('dst',  dst) 
cv2.waitKey()
cv2.destroyAllWindows()


corners.shape= (5, 1, 2)
corners= [[[387. 375.]]

 [[169. 375.]]

 [[265. 126.]]

 [[168. 270.]]

 [[266. 168.]]]
corners2.shape= (5, 1, 2)
corners2= [[[387. 375.]]

 [[169. 375.]]

 [[265. 126.]]

 [[387. 169.]]

 [[108. 269.]]]


In [77]:
# 0807.py
import cv2
import numpy as np

#1
src = cv2.imread('./data/chessBoard.jpg')
gray = cv2.cvtColor(src, cv2.COLOR_BGR2GRAY)
patternSize = (6, 3)
found, corners = cv2.findChessboardCorners(src, patternSize)
print('corners.shape=', corners.shape)

#2
term_crit=(cv2.TERM_CRITERIA_EPS+cv2.TERM_CRITERIA_MAX_ITER, 10, 0.01)
corners2 = cv2.cornerSubPix(gray, corners, (5,5), (-1,-1), term_crit)

#3
dst = src.copy()
cv2.drawChessboardCorners(dst, patternSize, corners2, found)

cv2.imshow('dst',  dst) 
cv2.waitKey()
cv2.destroyAllWindows()


corners.shape= (18, 1, 2)


In [85]:
# 0812.py
import cv2
import numpy as np

#1
src = cv2.imread('./data/banana.jpg')
gray = cv2.cvtColor(src, cv2.COLOR_BGR2GRAY)
ret, bImage = cv2.threshold(gray, 220, 255, cv2.THRESH_BINARY_INV)
##bImage = cv2.erode(bImage, None)
bImage = cv2.dilate(bImage, None)
##cv2.imshow('src',  src)
cv2.imshow('bImage',  bImage)

#2
mode   = cv2.RETR_EXTERNAL
method = cv2.CHAIN_APPROX_SIMPLE
image, contours, hierarchy = cv2.findContours(bImage, mode, method)
print('len(contours)=', len(contours))

maxLength = 0
k = 0
for i, cnt in enumerate(contours):
    perimeter = cv2.arcLength(cnt, closed = True)
    if perimeter> maxLength:
        maxLength = perimeter
        k = i
print('maxLength=', maxLength)
cnt = contours[k]
dst2 = src.copy()
cv2.drawContours(dst2, [cnt], 0, (255,0,0), 3)
cv2.imshow('dst2',  dst2)

#3
area = cv2.contourArea(cnt)
print('area=', area)
x, y, width, height = cv2.boundingRect(cnt)
dst3 = dst2.copy()
cv2.rectangle(dst3, (x, y), (x+width, y+height), (0,0,255), 2)
cv2.imshow('dst3',  dst3)

#4
rect = cv2.minAreaRect(cnt)
box = cv2.boxPoints(rect)
box = np.int32(box)
print('box=', box)
dst4 = dst2.copy()
cv2.drawContours(dst4,[box],0,(0,0,255),2)
cv2.imshow('dst4',  dst4)

#5
(x,y),radius = cv2.minEnclosingCircle(cnt)
dst5 = dst2.copy()
cv2.circle(dst5,(int(x),int(y)),int(radius),(0,0,255),2)
cv2.imshow('dst5',  dst5)

cv2.waitKey()
cv2.destroyAllWindows()


len(contours)= 1
maxLength= 936.0630499124527
area= 24663.0
box= [[275 391]
 [ 48 147]
 [159  44]
 [386 288]]


In [86]:
# 0813.py
import cv2
import numpy as np

#1
src = cv2.imread('./data/banana.jpg')
gray = cv2.cvtColor(src, cv2.COLOR_BGR2GRAY)
ret, bImage = cv2.threshold(gray, 220, 255, cv2.THRESH_BINARY_INV)
##bImage = cv2.erode(bImage, None)
bImage = cv2.dilate(bImage, None)
##cv2.imshow('src',  src)
##cv2.imshow('bImage',  bImage)

mode   = cv2.RETR_EXTERNAL
method = cv2.CHAIN_APPROX_SIMPLE
image, contours, hierarchy = cv2.findContours(bImage, mode, method)

dst = src.copy()
##cv2.drawContours(dst, contours, -1, (255,0,0), 3)
cnt = contours[0]
cv2.drawContours(dst, [cnt], 0, (255,0,0), 3)

#2
dst2 = dst.copy()
rows,cols = dst2.shape[:2]
[vx,vy,x,y] = cv2.fitLine(cnt, cv2.DIST_L2, 0, 0.01, 0.01)
y1 =  int((-x*vy/vx) + y)
y2 = int(((cols-x)*vy/vx)+y)
cv2.line(dst2,(0,y1), (cols-1,y2),(0,0,255), 2)
cv2.imshow('dst2',  dst2)

#3
ellipse = cv2.fitEllipse(cnt)
dst3 = dst.copy()
cv2.ellipse(dst3, ellipse,(0,0,255),2)
cv2.imshow('dst3',  dst3)

#4
poly = cv2.approxPolyDP(cnt, epsilon=20, closed=True)
dst4 = dst.copy()
cv2.drawContours(dst4, [poly], 0, (0,0,255), 2)
cv2.imshow('dst4',  dst4)

#5
dst5 = dst.copy()
for y in range(rows):
    for x in range(cols):
        if cv2.pointPolygonTest(cnt, (x, y), False)>0:
            dst5[y, x] = (0, 255, 0)            
cv2.imshow('dst5',  dst5)

cv2.waitKey()
cv2.destroyAllWindows()


In [104]:
# 0816.py
import cv2
import numpy as np

#1
ref_src  = cv2.imread('./data/refShapes.jpg')
ref_gray = cv2.cvtColor(ref_src, cv2.COLOR_BGR2GRAY)
ret, ref_bin = cv2.threshold(ref_gray, 0, 255,
                             cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)

##test_src  = cv2.imread('./data/testShapes1.jpg')
##test_src  = cv2.imread('./data/testShapes2.jpg')
test_src  = cv2.imread('./data/testShapes3.jpg')
test_gray = cv2.cvtColor(test_src, cv2.COLOR_BGR2GRAY)
ret, test_bin = cv2.threshold(test_gray, 0, 255,
                             cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)
mode   = cv2.RETR_EXTERNAL
method = cv2.CHAIN_APPROX_SIMPLE
image, ref_contours, hierarchy = cv2.findContours(ref_bin, mode, method)
image,test_contours, hierarchy = cv2.findContours(test_bin, mode, method)

#2
ref_dst = ref_src.copy()
colors = ((0,0,255), (0,255,0), (255,0,0))
for i, cnt in enumerate(ref_contours):
    cv2.drawContours(ref_dst, [cnt], 0, colors[i], 2)

#3: shape matching
test_dst = test_src.copy()
method = cv2.CONTOURS_MATCH_I1    
for i, cnt1 in enumerate(test_contours):
    matches = []
    for cnt2 in ref_contours:
        ret = cv2.matchShapes(cnt1, cnt2, method, 0)
        matches.append(ret)
    k = np.argmin(matches)
    cv2.drawContours(test_dst, [cnt1], 0, colors[k], 2)
       
cv2.imshow('ref_dst',  ref_dst)
cv2.imshow('test_dst', test_dst)

cv2.waitKey()
cv2.destroyAllWindows()


In [99]:
a=[[1,2,3],[4,5,6]]
a=np.array(a)

In [102]:
np.argwhere(a==3)

array([[0, 2]], dtype=int64)